This test is based upon `oarphpy_test/test_spark.py#test_spark_with_custom_library()` with small modifications to run in a Jupyter notebook.  You can run this notebook by itself using:
```
jupyter-nbconvert \
            --ExecutePreprocessor.timeout=3600 \
              --to notebook --execute --output /tmp/out \
                test_spark_ships_custom_library_in_notebook.ipynb
```

The notebook assumes that `oarphpy` and Spark are installed in the system / Jupyter kernel environment.  You can do that yourself using 
```
pip install oarphpy[spark]
```
(You may need to also install Java).

First, use the notebook to create a local python module we'd like to ship with our spark jobs.

In [ ]:
!rm -rf /tmp/test_spark_ships_custom_library_in_notebook || true
!mkdir -p /tmp/test_spark_ships_custom_library_in_notebook
import os; os.chdir('/tmp/test_spark_ships_custom_library_in_notebook')
!mkdir -p my_src_root/mymodule
!touch my_src_root/mymodule/__init__.py
!echo "pi = 3.14" > my_src_root/mymodule/foo.py

Test the local module

In [ ]:
!cd my_src_root && $(which python || which python3) -c 'from mymodule.foo import pi; print(pi)'

Set up and start the spark session

In [ ]:
import os
from oarphpy.spark import NBSpark
NBSpark.SRC_ROOT = '/tmp/test_spark_ships_custom_library_in_notebook/my_src_root/mymodule'
spark = NBSpark.getOrCreate()

Now test that the module gets shipped with the job in an egg

In [ ]:
def test_my_lib():
    # Make sure `mymodule` is not on the default local PYTHONPATH
    import os
    os.chdir('/tmp')
    
    import re
    import mymodule

    # The module should come from the included egg
    imp_path = mymodule.__file__
    assert re.match(
      r'^(.*)spark-(.*)/mymodule-0\.0\.0-py(.+)\.egg/mymodule/__init__\.py$',
      imp_path)

    # Now verify the module itself
    from mymodule import foo
    assert foo.pi == 3.14
    
    return True
  
# Now test that the lib gets egg-ified and shipped as a SparkFile
from oarphpy import spark as S
res = S.for_each_executor(spark, lambda: test_my_lib())
assert res and all(res)

`NBSpark` supports live library code edits (without restarting the kernel or Spark session).  Test that now.

In [ ]:
!echo "bar = 'baz'" > my_src_root/mymodule/foo.py

In [ ]:
def test_my_lib2():
    # Make sure `mymodule` is not on the default local PYTHONPATH
    import os
    os.chdir('/tmp')
    
    import re
    import mymodule

    # The module should come from the included egg
    imp_path = mymodule.__file__
    assert re.match(
      r'^(.*)spark-(.*)/mymodule-0\.0\.0-py(.+)\.egg/mymodule/__init__\.py$',
      imp_path)

    # Now verify the module itself
    from mymodule import foo
    assert foo.bar == 'baz'
    
    return True
  
# Now test that the updated lib is there
res = S.for_each_executor(spark, lambda: test_my_lib2())
assert res and all(res)